<a href="https://colab.research.google.com/github/breyer/tat/blob/main/TAT_EMA520_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data is based on the following backtest for the period from 1/1/2023 to 6/7/2023:**

https://tradeautomationtoolbox.com/byob-ticks/?save=eLwvpkg

I would love to get feedback ...



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

def filter_data(df, days):
    end_date = df['EntryTime'].max()
    start_date = end_date - pd.DateOffset(days=days)
    return df.loc[(df['EntryTime'] >= start_date) & (df['EntryTime'] <= end_date)].copy()

def rank_trading_days(df):
    profit_per_day = df.groupby('EntryDayOfWeek')['ProfitLossAfterSlippage'].sum()*100
    return profit_per_day.sort_values(ascending=False)

def rank_trading_times(df):
    df['EntryHourMinute'] = df['EntryTime'].dt.strftime('%H:%M')
    profit_per_time = df.groupby('EntryHourMinute')['ProfitLossAfterSlippage'].sum()*100
    return profit_per_time.sort_values(ascending=False)

def plot_heatmap(df, title, filename):
    df['EntryHour'] = df['EntryTime'].dt.hour
    df['EntryMinute'] = df['EntryTime'].dt.minute
    pivot_table = df.pivot_table(index='EntryHour', columns='EntryMinute', values='ProfitLossAfterSlippage', aggfunc='sum', fill_value=0)

    colors = sns.diverging_palette(10, 130, n=256).as_hex()
    colors = ['black'] + colors
    cmap = ListedColormap(colors)

    plt.figure(figsize=(10, 10))
    sns.heatmap(pivot_table, cmap=cmap, center=0, cbar=False)
    plt.title(title)
    plt.xlabel('Minute')
    plt.ylabel('Hour')
    plt.savefig(filename, format='png')
    plt.close()

def plot_heatmap_all_data(df, filename):
    df['EntryHour'] = df['EntryTime'].dt.hour
    df['EntryMinute'] = df['EntryTime'].dt.minute
    pivot_table = df.pivot_table(index='EntryHour', columns='EntryMinute', values='ProfitLossAfterSlippage', aggfunc='sum', fill_value=0)

    colors = sns.diverging_palette(10, 130, n=256).as_hex()
    colors = ['black'] + colors
    cmap = ListedColormap(colors)

    plt.figure(figsize=(5, 5))
    sns.heatmap(pivot_table, cmap=cmap, center=0)
    plt.title('Profit and Loss per Entry Time (Hour and Minute) - All Data')
    plt.xlabel('Minute')
    plt.ylabel('Hour')
    plt.savefig(filename, format='png')
    plt.close()

df = pd.read_csv("Trades.csv")
df['EntryTime'] = pd.to_datetime(df['EntryTime'])
df['EntryDate'] = df['EntryTime'].dt.date

days_list = [90, 60, 45, 30, 20, 15, 10, 5]

html_content = ""
for days in days_list:
    df_filtered = filter_data(df, days)
    filename = f'heatmap_{days}_days.png'
    plot_heatmap(df_filtered, f'Profit and Loss per Entry Time (Hour and Minute) - Last {days} Days', filename)
    html_content += f"<h3>Best times to enter a trade for the last {days} days:</h3>\n"
    html_content += f"<img src='{filename}' alt='Heatmap for {days} days'><br>\n"
    html_content += f"<table>{rank_trading_times(df).to_frame().reset_index().to_html(index=False)}</table>\n"

filename = 'heatmap_all_data.png'
plot_heatmap_all_data(df, filename)
html_content += "<h3>Best times to enter a trade for all data:</h3>\n"
html_content += f"<img src='{filename}' alt='Heatmap for all data'><br>\n"
html_content += f"<table>{rank_trading_times(df).to_frame().reset_index().to_html(index=False)}</table>\n"

df['EntryDayOfWeek'] = df['EntryTime'].dt.dayofweek
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['EntryDayOfWeek'] = df['EntryDayOfWeek'].apply(lambda x: days_of_week[x])

html_content += "<h3>Profit per day of the week:</h3>\n"
html_content += f"<p>{rank_trading_days(df)}</p>\n"


In [2]:
pivot_table = df.pivot_table(index='EntryDate', columns='EntryHourMinute', values='ProfitLossAfterSlippage', aggfunc='sum', fill_value=0)

colors = sns.diverging_palette(10, 130, n=256).as_hex()
colors = ['black'] + colors
cmap = ListedColormap(colors)

plt.figure(figsize=(20, 20))
sns.heatmap(pivot_table, cmap=cmap, center=0)
plt.title('Profit and Loss per Entry Time (Hour and Minute)')
plt.xlabel('Time')
plt.ylabel('Date')
plt.savefig("heatmap_date_time.png")
plt.close()

html_content += f"<h3>Heatmap of Profit and Loss per Entry Date and Time:</h3>\n"
html_content += f"<img src='heatmap_date_time.png' alt='Heatmap of Profit and Loss per Entry Date and Time'><br>\n"

In [3]:
pivot_table = df.pivot_table(index='EntryDayOfWeek', columns='EntryHourMinute', values='ProfitLossAfterSlippage', aggfunc='sum', fill_value=0)

colors = sns.diverging_palette(10, 130, n=256).as_hex()
colors = ['black'] + colors
cmap = ListedColormap(colors)

plt.figure(figsize=(20, 20))
sns.heatmap(pivot_table, cmap=cmap, center=0)
plt.title('Profit and Loss per Entry Time (Hour and Minute)')
plt.xlabel('Time')
plt.ylabel('Day of Week')
plt.savefig("heatmap_dayofweek_time.png")
plt.close()

html_content += f"<h3>Heatmap of Profit and Loss per Entry Day of Week and Time:</h3>\n"
html_content += f"<img src='heatmap_dayofweek_time.png' alt='Heatmap of Profit and Loss per Entry Day of Week and Time'><br>\n"

with open('heatmap_report.html', 'w') as f:
    f.write(html_content)